In [1]:
import numpy as np  
import scipy.io as sio
from scipy import linalg
import matplotlib.pyplot as plt

In [30]:
from scipy.io import loadmat
M = np.array(loadmat('M.mat')['M']).astype(int)
Mcounts = np.array(loadmat('Mcounts.mat')['Mcounts']).astype(int)
y = np.array(loadmat('y.mat')['y'])
words = np.array(loadmat('words.mat')['words']).flatten()

In [45]:
class1_bool = (y == 1).flatten()
class2_bool = (y == -1).flatten()
class1_freq = np.sum(M[class1_bool, :], axis=0)/len(class1_bool)
class2_freq = np.sum(M[class2_bool, :], axis=0)/len(class2_bool)
information_gain =np.abs(class1_freq - class2_freq)

In [86]:
np.sum(class2_bool)

71

In [48]:
def CUR(A, W, Vh, c, k, r):
    # A: input m x n array
    # W: The matrix of left singular vectors for A, m x rank(A) array
    # Vh: The matrix of right singular vectors for A, rankd(A) x n array
    # c, r: parameters for column/row selection likelihood
    # k: rank for choosing the rank-truncated SVD

    # Pick k highest left singular vectors
    Vk = Vh[0:k, :]
    Uk = W[:, 0:k]

    # Randomly select columns, rows with high statistical leverage
    [C, column_leverage] = columnselect(A, Vk, k, c)
    [R, row_leverage] = columnselect(A.T, Uk.T, k, r)
    R = R.T

    # Compute U
    Cinv = linalg.pinv(C)
    Rinv = linalg.pinv(R)
    U = np.dot(Cinv, np.dot(A, Rinv))

    return [C, U, R, column_leverage, row_leverage]

def columnselect(A, Vk, k, c):
    
    # Compute leverage scores
    leverage = (1/k)*np.sum(Vk**2, axis=0)
    
    # Randomyly pick columns to keep (according to leverage) 
    #keep_bool = np.zeros(A.shape[1], dtype=bool)
    uniform = np.random.uniform(0, 1, A.shape[1])
    keep_prob = np.minimum(1, c*leverage)
    keep_bool = uniform < keep_prob
        
    # If no columns chosen, choose the one with highest leverage
    if np.all(~keep_bool):
        j = np.argmax(leverage)
        keep_bool[j] = True

    Areduced = A[:, keep_bool]

    return [Areduced, leverage]


def getLeverage( W, Vh, k):
        # Pick k highest left singular vectors
    Vk = Vh[0:k, :]
    Uk = W[:, 0:k].T  
    # Compute leverage scores
    column_leverage = (1/k)*np.sum(Vk**2, axis=0)
    row_leverage = (1/k)*np.sum(Uk**2, axis=0)

    return [column_leverage, row_leverage]

In [62]:
print("The top 5 information gain words are:")
sorted_idx = np.argsort(-information_gain)
top5_ig_words = set([w[0] for w in words[sorted_idx[0:5]]])
print(top5_ig_words)

The top 5 information gain words are:
{'indiana', 'florida', 'south', 'miami', 'evansville'}


In [131]:
preM = M.copy()
preM = preM + 50*np.dot(y.transpose(), preM)/(np.linalg.norm(y)*np.linalg.norm(preM))
U,S,Vh = linalg.svd(preM,compute_uv=True,full_matrices=False)
col_leverage, row_leverage = getLeverage(U, Vh, 20)

In [132]:
lev_sorted_idx = np.argsort(-col_leverage)

top20_lev_words = set([w[0] for w in words[lev_sorted_idx[0:20]]])
print(top20_lev_words)

{'contact', 'welcome', 'south', 'miami', 'company', 'indiana', 'jpg', 'click', 'service', 'provide', '812', 'page', 'services', 'evansville', 'information', 'florida', 'the', 'phone', 'and', 'gif'}


In [133]:
top5_ig_words - top20_lev_words

set()